In [2]:
from utils import *
import numpy as np
import pickle


In [3]:
index_to_english_alphabet=pickle.load(open('vocab_tools/index_to_english_alphabet.pickle', 'rb'))
index_to_hindi_alphabet=pickle.load(open('vocab_tools/index_to_hindi_alphabet.pickle', 'rb'))

hindi_alphabet_to_index=pickle.load(open('vocab_tools/hindi_alphabet_to_index.pickle', 'rb')) 
english_alphabet_to_index=pickle.load(open('vocab_tools/english_alphabet_to_index.pickle', 'rb')) 

In [4]:
X_train=np.load('simple_data/X_train.npy')
X_valid=np.load('simple_data/X_val.npy')

y_train=np.load('simple_data/y_train.npy')
y_valid=np.load('simple_data/y_val.npy')

In [5]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

In [6]:
class Eng_Hind_Dataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, in_file, out_file, root_dir='simple_data',device='cuda'):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.input = torch.tensor(np.load(root_dir+'/'+in_file))
        self.output = torch.tensor(np.load(root_dir+'/'+out_file))
        
        assert(len(self.input)==len(self.output),"Error: I/O Lengths must be same")
        

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        X=self.input[idx]
        X=X.to(device)
        y=self.output[idx]
        y=y.to(device)
        


        sample = {'input': X, 'output': y}

        return sample

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
device

device(type='cuda')

In [9]:
training_data=Eng_Hind_Dataset("X_train.npy","y_train.npy",device=device)
val_data=Eng_Hind_Dataset("X_val.npy","y_val.npy",device=device)
test_data=Eng_Hind_Dataset("X_test.npy","y_test.npy",device=device)

In [10]:
train_dataloader = DataLoader(training_data, batch_size=16,shuffle=True)

In [11]:
val_dataloader = DataLoader(val_data, batch_size=16,shuffle=True)

In [12]:
import torch.nn as nn

In [13]:
def cell_type(mode:str='rnn'):
    mode=mode.lower()
    if mode == 'rnn':
        return nn.RNN
    elif mode =='gru':
        return nn.GRU
    else:
        return nn.LSTM
        

In [14]:
class Encoder(nn.Module):


    def __init__(self, input_size, embed_size, enc_hid_size, dec_hid_size, num_layers, cell_mode, dropout, is_bi):
        super().__init__()
        
        # create embedding layer
        self.embedding = nn.Embedding(input_size, embed_size,padding_idx=english_alphabet_to_index['.'])

        #creating LSTM/GRU/RNN cell
        cell=cell_type(cell_mode)
        
        self.cell=cell(embed_size,enc_hid_size,num_layers,dropout=dropout,bidirectional=is_bi,batch_first=True)
        self.cell_mode=cell_mode
        self.is_bi=is_bi
        if is_bi:
            self.fc=nn.Linear(enc_hid_size*2, dec_hid_size)
        else:
            self.fc=nn.Linear(enc_hid_size, dec_hid_size)
        
        

    def forward(self, input_batch: torch.LongTensor):
        
        embedded = self.embedding(input_batch) # [sent len, batch size, emb dim]
        
        if self.cell_mode.lower()=='lstm':
            outputs, (hidden, cell) = self.cell(embedded)
            
            if self.is_bi:
                concated = torch.cat((hidden[ :,-2, :], hidden[ :,-1, :]), dim=1)
            else:
                concated = hidden[ :,-1, :]
            
            hidden = torch.tanh(self.fc(concated))
            return outputs,hidden
             
        
        else:
            outputs, hidden = self.cell(embedded)           
            if self.is_bi:
                concated = torch.cat((hidden[ -2, :,:], hidden[ -1, :,:]), dim=1)
                
            else:
                concated = hidden[ -1,:, :]
                
            hidden = torch.tanh(self.fc(concated))
            
            return outputs,hidden

        


In [15]:
class Attention(nn.Module):

    def __init__(self, enc_hid_dim, dec_hid_dim, is_bi):
        super().__init__()
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        if is_bi:
            self.fc1 = nn.Linear(enc_hid_dim * 2 + dec_hid_dim, dec_hid_dim)
        else:
            self.fc1 = nn.Linear(enc_hid_dim + dec_hid_dim, dec_hid_dim)
            

            
        self.fc2 = nn.Linear(dec_hid_dim, 1, bias=False)

    def forward(self, encoder_outputs, hidden):
        src_len = encoder_outputs.shape[1]
        batch_size = encoder_outputs.shape[0]
        
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        outputs = encoder_outputs
        
        concat = torch.cat((hidden, outputs), dim=2)
        energy = torch.tanh(self.fc1(concat))

        attention = self.fc2(energy).squeeze(dim=2)        
        attention_weight = torch.softmax(attention, dim=1)
        return attention_weight

In [16]:
class Decoder(nn.Module):

    def __init__(self, output_size, embed_size, enc_hid_dim, dec_hid_dim, num_layers, cell_mode,\
                 dropout, attention, is_bi):
        super().__init__()
    
        self.dropout = dropout
        self.attention = attention
        self.output_size=output_size

        self.embedding = nn.Embedding(output_size, embed_size,padding_idx=hindi_alphabet_to_index['.'])
        cell=cell_type(cell_mode)
        
        
        if is_bi:
            self.cell=cell((enc_hid_dim * 2) + embed_size, dec_hid_dim,num_layers,\
                           dropout=dropout,bidirectional=False,batch_first=False)
        

        else:
            self.cell=cell(enc_hid_dim + embed_size, dec_hid_dim,num_layers,\
                           dropout=dropout,bidirectional=False,batch_first=False)
            
        
        
        self.out = nn.Linear(dec_hid_dim, output_size)

    def forward(self, trg, encoder_outputs, hidden):
 
        attention = self.attention(encoder_outputs, hidden).unsqueeze(1)
        context = torch.bmm(attention, encoder_outputs).permute(1, 0, 2)

        # input sentence -> embedding
        # [1, batch size, emb dim]
        embedded = self.embedding(trg.unsqueeze(0))
        cell_input = torch.cat((embedded, context), dim=2)

        outputs, hidden = self.cell(cell_input, hidden.unsqueeze(0))
        prediction = self.out(outputs.squeeze(0))
        return prediction, hidden.squeeze(0)

In [17]:
ss1=next(iter(train_dataloader))

In [18]:
hindi_alphabet_to_index['.']

2

In [58]:

encoder = Encoder(30, embed_size=64, enc_hid_size=128, dec_hid_size=12, num_layers=1, cell_mode='gru',\
                  dropout=0.2, is_bi=False).to(device)
outputs, hidden = encoder(ss1['input'])

attention = Attention(enc_hid_dim=128, dec_hid_dim=12, is_bi=False).to(device)


In [59]:
hidden.shape

torch.Size([16, 12])

In [60]:
outputs.shape

torch.Size([16, 30, 128])

In [61]:
decoder = Decoder(68, embed_size=128, enc_hid_dim=128, dec_hid_dim=12, num_layers=1,cell_mode='gru',\
                  dropout=0.2, attention=attention, is_bi=False).to(device)
prediction, decoder_hidden = decoder(ss1['output'][:,0], outputs, hidden)

prediction.shape, decoder_hidden.shape

(torch.Size([16, 68]), torch.Size([16, 12]))

In [21]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device


    def forward(self, source_batch, target_batch, teacher_forcing_ratio=0.5):

        batch_size,max_len  = target_batch.shape
        target_vocab_size = self.decoder.output_size

        outputs = torch.zeros(max_len, batch_size, target_vocab_size).to(self.device)

        # last hidden & cell state of the encoder is used as the decoder's initial hidden state
        e_outs,hidden = self.encoder(source_batch)       

        trg = target_batch[:,0]
        for i in range(1, max_len):
            prediction, hidden = self.decoder(trg,e_outs, hidden)
            outputs[i] = prediction

            if np.random.random() < teacher_forcing_ratio:
                trg = target_batch[:,i]
            else:
                trg = prediction.argmax(1)

        return outputs


        
        

In [22]:
State_bi=False
encoder = Encoder(30, 128, 128, 128, 1, 'gru', 0.2, State_bi).to(device)

attention = Attention(128, 128, State_bi).to(device)

decoder = Decoder(68, 128, 128, 128, 1,'gru', 0.2, attention, State_bi).to(device)

In [23]:
def accuracy_calc(target_seq,seq2,mode='full',device=device):# predicted
    eos_index=(target_seq==hindi_alphabet_to_index['>']).nonzero()
    eos_idx=eos_index[:,1]
    
    correct=torch.Tensor([0]).to(device)
    correct_chars=torch.Tensor([0]).to(device)
    tot_chars=torch.Tensor([0]).to(device)
    for iterate,idx in enumerate(eos_idx):
        inputter=seq2[iterate][:idx]
        outputter=target_seq[iterate][:idx]
        if torch.all(torch.eq(inputter,outputter)):
            correct+=1
            correct_chars+=idx
            tot_chars+=idx
        else:
            correct_chars+=torch.sum(inputter == outputter).item()
            tot_chars+=idx
            
#         print(correct,correct_chars,tot_chars)
        
    return correct.item(),correct_chars.item(),tot_chars.item()
            
            
        
    

In [24]:
accuracy_calc(ss1['output'],ss1['output'])

(16.0, 152.0, 152.0)

In [25]:
def train(seq2seq, iterator, optimizer, criterion):
    
    
    seq2seq.train()
    
    epoch_loss = 0
    correct=0
    correct_char=0
    tot_char=0
    
    relax_acc=0
    
    
    for batch in iterator:
        optimizer.zero_grad()
        outputs = seq2seq(batch['input'], batch['output'])
        batch_label=batch['output'].transpose(0,1)
        batch_size=len(batch['output'])
        
        _, predicted = torch.max(outputs, dim=2)
        outputs_flatten = outputs.view(-1, outputs.shape[-1])
        trg_flatten = batch_label.reshape(-1)
        

        trg_flatten.requires_grad=False
        loss = criterion(outputs_flatten, trg_flatten)
        correct_temp,correct_chars_temp,tot_chars_temp=accuracy_calc(batch['output'],predicted.transpose(0,1))
        
        #___________
        
        correct+=correct_temp
        correct_char+=correct_chars_temp
        tot_char+=tot_chars_temp
        
        
        #_______________
        

        loss.backward()
        optimizer.step()
        

        epoch_loss += loss.item()
        


    return epoch_loss / len(iterator), correct/(len(iterator)*16),correct_char/tot_char

In [26]:
def evaluate(seq2seq, iterator, criterion):
    seq2seq.eval()

    epoch_loss = 0
    correct=0
    correct_char=0
    tot_char=0
    
    relax_acc=0
    
    with torch.no_grad():
        for batch in iterator:
            outputs = seq2seq(batch['input'], batch['output'],teacher_forcing_ratio=0)
            batch_label=batch['output'].transpose(0,1)
            batch_size=len(batch['output'])


            _, predicted = torch.max(outputs, dim=2)
            #print('wow_preds',predicted.shape)

            outputs_flatten = outputs.view(-1, outputs.shape[-1])
            trg_flatten = batch_label.reshape(-1)

            loss = criterion(outputs_flatten, trg_flatten)
            
            correct_temp,correct_chars_temp,tot_chars_temp=accuracy_calc(batch['output'],predicted.transpose(0,1))
        
            #___________

            correct+=correct_temp
            correct_char+=correct_chars_temp
            tot_char+=tot_chars_temp

            #_______________       
            
            epoch_loss += loss.item()
            

    return epoch_loss / len(iterator), correct/(len(iterator)*16),correct_char/tot_char



In [27]:
def epoch_time(start_time, end_time):
    e_time = end_time - start_time
    mins = e_time // 60
    secs = e_time%60
    return mins, secs,

In [28]:
def count_params(model):
    return sum(param.numel() for param in model.parameters() if param.requires_grad)

In [29]:
import time
import random

In [63]:
import torch.optim as optim
def make_model(train_iterator,valid_iterator, enc_embed_size, enc_hid_size, dec_embed_size, dec_hid_size,\
               num_layers, cell_mode,\
                 dropout, is_bi, epochs=20):
    
    
    E = Encoder(30,  embed_size=enc_embed_size, enc_hid_size=enc_hid_size, dec_hid_size=dec_hid_size,\
                num_layers=num_layers, cell_mode=cell_mode, dropout=dropout, is_bi=is_bi).to(device)
    
    
    A = Attention(enc_hid_dim=enc_hid_size, dec_hid_dim=dec_hid_size, is_bi=is_bi).to(device)

    D = Decoder(68, embed_size=dec_embed_size, enc_hid_dim=enc_hid_size, dec_hid_dim=dec_hid_size,\
                num_layers=num_layers, cell_mode=cell_mode,\
                 dropout=dropout, attention=attention, is_bi=is_bi).to(device)

    S=Seq2Seq(E,D,device)
    S.to(device)    
    print(f'The model has {count_params(S):,} trainable parameters')
    
    optimizer = optim.Adam(S.parameters())
    criterion = nn.CrossEntropyLoss(ignore_index=hindi_alphabet_to_index['.'])
    criterion=criterion.to(device)
    
    best_valid_loss = float('inf')
#     return S

    for epoch in range(epochs):    
        start_time = time.time()
        train_loss,train_acc,train_stuff = train(S, train_iterator, optimizer, criterion)
        valid_loss,valid_acc,val_stuff = evaluate(S, valid_iterator, criterion)
        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(S.state_dict(), 'model1.pt')

        # it's easier to see a change in perplexity between epoch as it's an exponential
        # of the loss, hence the scale of the measure is much bigger
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs:.2f}s')
        print(f'\t Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
        print(f'\t Relaxed Train. Acc: {train_stuff*100:.2f}% | Relaxed Val. Acc: {val_stuff*100:.2f}%')
        
    return S

    

In [68]:
train_iterator=train_dataloader
valid_iterator=val_dataloader
SS=make_model(train_iterator,valid_iterator,enc_embed_size=128, enc_hid_size=128, dec_embed_size=128, dec_hid_size=256,\
               num_layers=1, cell_mode='gru',\
                 dropout=0.27, is_bi=True, epochs=20)

The model has 788,716 trainable parameters


RuntimeError: mat1 and mat2 shapes cannot be multiplied (480x512 and 140x12)

In [ ]:
a=next(iter(train_dataloader))

In [ ]:
bya=SS(a['input'],a['output'])

In [ ]:
a['output'].shape

In [ ]:
bya.shape

In [ ]:
import math

In [ ]:
def word_from_torchies(torchie1,index_toalp):
    torchie=torchie1.cpu().numpy()
    return word_from_vecs(torchie,index_toalp,False)

In [ ]:
def word_from_batch(batch):
    wordlet=[]
    for i in range(len(batch)):
        wordlet.append(word_from_torchies(batch[i],index_to_hindi_alphabet))
    return wordlet
        

In [ ]:
word_from_torchies(bya.transpose(0,1).argmax(2)[0],index_to_hindi_alphabet)

In [ ]:
index_to_hindi_alphabet[65]

In [ ]:
def make_wrd(stuff):
    asa=[]
    for k in stuff.cpu().numpy():
        asa.append(index_to_hindi_alphabet[k])
    return "".join(asa)
    

In [ ]:
make_wrd(bya.transpose(0,1).argmax(2)[10])

In [ ]:
bya.transpose(0,1).argmax(2)[10]

In [ ]:
for i in range(16):
    print('................')
    print(i)
    print(word_from_torchies(a['output'][i],index_to_hindi_alphabet),\
         '---',\
         make_wrd(bya.transpose(0,1).argmax(2)[i])\
         )


In [ ]:
word (self, source_batch,target_batch):
    max_len, batch_size = target_batch.shape
    outputs = torch.zeros(max_len, batch_size, target_vocab_size).to(self.device)


            # last hidden & cell state of the encoder is used as the decoder's initial hidden state
    hidden, cell = self.encoder(source_batch)

    wordet=[]


    trg = torch.tensor(hindi_alphabet_to_index['<'])
    trg=trg.to(device)
    wordet.append(index_to_hindi_alphabet(trg.cpu().numpy()))
    for i in range(1, max_len):
        prediction, hidden, cell = self.decoder(trg, hidden, cell)
        outputs[i] = prediction
        trg = prediction.argmax(1)
        wordet.append(index_to_hindi_alphabet(trg.cpu().numpy()))


    return ''.join(wordet)

In [ ]:
word_from_vecs(X_valid[0],index_to_english_alphabet,False)

In [ ]:
word_from_vecs(y_valid[0],index_to_hindi_alphabet,False)

In [ ]:
next(iter(train_dataloader))

In [ ]:
            for j in range(predicted.shape[1]):
                predicted_seq = predicted[:, j]
                targets_seq = target_seq[:, j]

                # Find the index of the first EOS token in the sequence
                eos_idx = (targets_seq == hin_token_map["\n"]).nonzero()
                if eos_idx.numel() > 0:
                    eos_idx = eos_idx[0][0]
                    predicted_seq = predicted_seq[:eos_idx]
                    targets_seq = targets_seq[:eos_idx]

In [ ]:
s